In [35]:
import wikipedia
import os
import json
import tqdm
import argparse

import multiqa_utils.general_utils as gu
import multiqa_utils.qampari_utils as qu
import multiqa_utils.retrieval_utils as ru
import multiqa_utils.wikipedia_utils as wu
import multiqa_utils.distributed_utils as du

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## First, build the ent_string to wikipage_title cache

In [68]:
path_args = gu.current_default_path_args()

In [43]:
all_titles = wu.build_gt_wikititle_set(path_args, force=False)

>> Index already exists: /scratch/ddr8143/wikipedia/tagme_dumps_qampari_wikipedia/postprocessed/gt_title_set.json


In [44]:
curr_cache = wu.get_initial_str2wikipage_cache(all_titles, path_args, force=False)

>> Loading the cache
>> Initial cache size: 6325468


In [45]:
# Without any flags set true (add_elq, add_gpt, add_wikitags) this just
#   goes through the current list and removes strings already in cache.
ru.aggregate_strs_to_add_to_cache(
    path_args,
    use_tqdm=True,
    curr_cache=curr_cache,
)

>> Load existing string list: /scratch/ddr8143/wikipedia/tagme_dumps_qampari_wikipedia/postprocessed/strs_to_add_to_cache_v0.json
>> Initial string list length: 3971529
>> Removing strings already in cache
>> New string list length: 3971529
>> Writing file
>> Dumped to: /scratch/ddr8143/wikipedia/tagme_dumps_qampari_wikipedia/postprocessed/strs_to_add_to_cache_v0.json


In [46]:
strs_to_add = json.load(open(path_args.strs_for_cache_path))

In [50]:
du.distributed_build_str2wikipage_cache(
    path_args,
    job_id=0,
    total_num_jobs=40000,
    all_strs_to_add=strs_to_add,
    use_tqdm=True,    
)

>> This job gets from 0 to 100 of full list length 3971529 bc its job 0 of 40000
>> Index already exists: /scratch/ddr8143/wikipedia/tagme_dumps_qampari_wikipedia/postprocessed/gt_title_set.json
>> Loading the cache
>> Initial cache size: 6325468
>> Adding new strings to cache: 100


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.47it/s]


>> Final cache size: 6325557
>> Dumped: /scratch/ddr8143/wikipedia/tagme_dumps_qampari_wikipedia/postprocessed/str2wikipage_cache.json__job0
>> Dumped: /scratch/ddr8143/wikipedia/tagme_dumps_qampari_wikipedia/postprocessed/str2wikipage_disambig_cache.json__job0


In [65]:
du.aggregate_checkpoint_dicts(path_args.cache_path, remove_processed=False)

>> Only one file, skipping aggregation


In [66]:
du.aggregate_checkpoint_dicts(path_args.disambig_cache_path, remove_processed=False)

>> Only one file, skipping aggregation


## Previously Ran
But haven't retested after the refactor.